<a href="https://colab.research.google.com/github/lbdlab/NGS-LBD/blob/master/notebooks/FastRPembeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install neo4j pandas seaborn matplotlib

     |████████████████████████████████| 81kB 3.7MB/s 
  Created wheel for neo4j: filename=neo4j-4.3.1-cp37-none-any.whl size=99332 sha256=99479e30b52f066b17d89051a4b0825b8388c8441d55cbafb1b349145adca383
  Stored in directory: /root/.cache/pip/wheels/23/13/72/0cc2405898bd9a7baef6512df3abf83873da9ba48c04acc818
Successfully built neo4j


In [3]:

%matplotlib inline
from neo4j import GraphDatabase
import pandas as pd
import seaborn as sns
from matplotlib import pyplot
#Test asdadasd
sns.set(font_scale = 1.5)

In [4]:
# Connect to Neo4j
driver = GraphDatabase.driver('neo4j://212.235.239.171:22333', auth=('neo4j', 'ngslbd2020'))

def read_query(query, params={}):
    with driver.session(database="semmed43cord19") as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [ ]:
read_query("""
MATCH (c:Concept) RETURN c.name LIMIT 5
""")

,c.name
0,Radiation Damage
1,Malus xiaojinensis
2,Bicytopenia
3,Deriphat
4,Granulomatous myositis


In [ ]:
read_query("""
MATCH (c:Concept) WHERE c.name = $name
RETURN c.name
""", {'name':'Radiation Damage'})

,c.name
0,Radiation Damage


In [ ]:
read_query("""
CALL gds.graph.create('fastrp', ['dsyn','phsu'], {TREATS:{orientation:'UNDIRECTED'}},
  {nodeProperties:['arg_inst_freq', 'arg_rel_freq'], relationshipProperties:['freq']})
""")

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,createMillis
0,{'phsu': {'properties': {'arg_rel_freq': {'pro...,"{'TREATS': {'orientation': 'UNDIRECTED', 'aggr...",fastrp,48503,388282,1035


In [ ]:
read_query("""
CALL gds.beta.fastRPExtended.write(
  'fastrp',
  {writeProperty:'fastrp',
   relationshipWeightProperty:'freq',
   featureProperties: ['arg_inst_freq', 'arg_rel_freq'],
   embeddingDimension: 128
   }
)
""")

,nodeCount,nodePropertiesWritten,createMillis,computeMillis,writeMillis,configuration
0,48503,48503,1,626,1807,"{'writeConcurrency': 4, 'relationshipWeightPro..."


In [ ]:
read_query("""
MATCH (d:dsyn)
RETURN d.fastrp as embedding
LIMIT 5
""")

,embedding
0,"[0.08636206388473511, -0.22889243066310883, 0...."
1,"[0.08845087885856628, -0.28754985332489014, 0...."
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
# seems that isolated nodes have 0.0 embeddings all the way
# also other concepts than just phsu can treat diseases

Drug repurposing with Cosine similarity
- Find top 5 existing drugs for headaches by TREATS frequency
- Find top 10 similar drugs for each existing drugs that don't treat headache
- Return top 10 candidates

Drug repurposing with Cosine similarity
- Find top 5 existing drugs for headaches by TREATS frequency
- Find top 10 similar drugs for each existing drugs that don't treat headache
- Return top 10 candidates

In [12]:
read_query("""
// find the top five existing/mentioned drugs for treating headache
MATCH (c:Concept{name:"Headache"})<-[t:TREATS]-(existing:phsu)
WITH c,existing, t.freq as frequency
ORDER BY frequency DESC LIMIT 5
// find the top 10 similar drugs for each existing drug
CALL {
WITH existing
MATCH (p:phsu)
WHERE exists (p.fastrp) AND NOT (:Concept{name:"Headache"})<-[:TREATS]-(p:phsu)
WITH existing, p, gds.alpha.similarity.cosine(existing.fastrp, p.fastrp) AS similarity
WITH p, similarity as sim
ORDER BY sim DESC LIMIT 10
RETURN p as t, sim
}
// return candidates
WITH t, sum(sim) as score
ORDER BY score
DESC LIMIT 10
RETURN t.name as possible, score""")

,possible,score
0,Imitrex,1.995303
1,GR-43175,1.995303
2,Isometheptene mucate,1.995303
3,rizatriptan benzoate,1.995303
4,MK 0462,1.995303
5,BMS 180048,1.995303
6,rofecoxib,1.873681
7,meloxicam,1.850041
8,diclofenac,1.830451
9,tiaprofenic acid,1.823212


Drug repurposing with Euclidean similarity
- Find top 5 existing drugs for headaches by TREATS frequency
- Find top 10 similar drugs for each existing drugs that don't treat headache
- Return top 10 candidates

In [10]:
read_query("""
// find the top five existing/mentioned drugs for treating headache
MATCH (c:Concept{name:"Headache"})<-[t:TREATS]-(existing:phsu)
WITH c,existing, t.freq as frequency
ORDER BY frequency DESC LIMIT 5
// find the top 10 similar drugs for each existing drug
CALL {
WITH existing
MATCH (p:phsu)
WHERE exists (p.fastrp) AND NOT (:Concept{name:"Headache"})<-[:TREATS]-(p:phsu)
WITH existing, p, gds.alpha.similarity.euclideanDistance(existing.fastrp, p.fastrp) AS similarity
WITH p, similarity as sim
ORDER BY sim DESC LIMIT 10
RETURN p as t, sim
}
// return candidates
WITH t, sum(sim) as score
ORDER BY score
DESC LIMIT 10
RETURN t.name as possible, score""")

,possible,score
0,"Mercury, Ammoniated",11.063050
1,Enclomiphene Citrate,8.776846
2,semduramicin,6.537046
3,Innohep,4.505249
4,GMI-1271,4.502101
5,napsagatran,4.502101
6,Lovenox,4.502101
7,SR 90107A-ORG 31540,4.502101
8,Abbokinase,4.502101
9,idraparinux,4.424000


# Testing virtual graph with ISA relationships

ISA relationships are directed, and follow hierarchy

Example: Simvastatin - ISA -> Statin

In [15]:
#Checking the count of the virtual relationships
read_query("""
MATCH (source)<-[a:ISA*0..1]-(:phsu)-[r:TREATS]->(:dsyn)-[b:ISA*0..1]->(target)
  WHERE r.freq > 1 AND all(x IN a WHERE x.freq > 1) AND all(x IN b WHERE x.freq > 1)
  RETURN count(*)
""")

,count(*)
0,6748601


In [11]:
# Exclude ISA

read_query("""
// find the top five existing/mentioned drugs for treating headache
MATCH (c:Concept{name:"Headache"})<-[t:TREATS]-(existing:phsu)
WITH c,existing, t.freq as frequency
ORDER BY frequency DESC LIMIT 5
// find the top 10 similar drugs for each existing drug
CALL {
WITH existing
MATCH (p:phsu)
WHERE exists (p.fastrp) AND NOT (:Concept{name:"Headache"})<-[:TREATS]-(p:phsu) AND NOT (:Concept{name:"Headache"})<-[:TREATS]-(:phsu)-[:ISA]-(p)
WITH existing, p, gds.alpha.similarity.euclideanDistance(existing.fastrp, p.fastrp) AS similarity
WITH p, similarity as sim
ORDER BY sim DESC LIMIT 10
RETURN p as t, sim
}
// return candidates
WITH t, sum(sim) as score
ORDER BY score
DESC LIMIT 10
RETURN t.name as possible, score""")

,possible,score
0,"Mercury, Ammoniated",11.063050
1,Enclomiphene Citrate,8.776846
2,semduramicin,6.537046
3,Innohep,4.505249
4,napsagatran,4.502101
5,Lovenox,4.502101
6,GMI-1271,4.502101
7,SR 90107A-ORG 31540,4.502101
8,Abbokinase,4.502101
9,idraparinux,4.424000


In [ ]:
# Seems at least ok... Imitrex is known to treat headaches

In [5]:
# New version of embeddings with ISA relationship

In [ ]:
read_query("""
CALL gds.graph.create.cypher("is_a",
// load all nodes that treat dsyn or are ISA related to nodes that treat dsyn

"MATCH (n)-[:ISA*0..1]-(r:phsu)-[t:TREATS]->(:dsyn)
 WHERE t.freq > 1
 RETURN distinct id(n) as id, n.arg_inst_freq as arg_inst_freq, n.arg_rel_freq as arg_rel_freq
 UNION
 MATCH (d:dsyn)-[:ISA*0..1]-(target) RETURN id(target) as id, target.arg_inst_freq as arg_inst_freq, target.arg_rel_freq as arg_rel_freq",




 // load all relationship between phsu and dsyn and include all ISA variants
 "MATCH (source)-[:ISA*0..1]-(:phsu)-[r:TREATS]->(:dsyn)-[:ISA*0..1]-(target)
  WHERE r.freq > 1
  RETURN distinct id(source) as source, id(target) as target, r.freq as freq"



 )
""")

In [9]:
read_query("""
MATCH (source)-[a:ISA*0..1]-(:phsu)-[r:TREATS]->(:dsyn)-[b:ISA*0..1]-(target)
  WHERE r.freq > 1 AND a.freq > 1 AND b.freq > 1
  RETURN count(*)
""")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



CypherSyntaxError: ignored

In [ ]:
tažko 